# Automating a processing chain using Python apps (scripts)
So far, we have run Jupyter Notebooks to go through the steps involved in the creation of a baseline image composite, the downloading and processing of change detection images, the classification into land cover maps, the change detection and verification, and the aggregation of many change maps into one report image file. We have then vectorised the report image to reduce its file size and emailed the reports to registered users as an email attachment.
This notebook shows how we can build an efficient and more automated forest alerts processing chain. We will do this by basically turning the four previous notebooks into Python scripts (also sometimes called Apps). A Python script ends with .py and is executable from within Python.

# Setup: Requirements to use this Notebook

Check which directory we are in.

In [1]:
pwd

'/home/cmsstudent/pyeo/notebooks'

Go to the pyeo home directory.

In [2]:
import os
pyeo_dir = '/home/cmsstudent/pyeo'
os.chdir(pyeo_dir)
workdir = os.getcwd()
print(workdir)
config_path = os.path.join(pyeo_dir, 'pyeo_linux_azure.ini')
print(config_path)

/home/cmsstudent/pyeo
/home/cmsstudent/pyeo/pyeo_linux_azure.ini


We did this in the previous notebook step-by-step. Here, we initialie the notebook in one code cell to speed up the process.

In [3]:
import argparse
import configparser
import cProfile
import datetime
from datetime import datetime
import email
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.message import EmailMessage
import geopandas as gpd
import glob
import json
import numpy as np
import pandas as pd
import os
from osgeo import gdal
import shutil
import smtplib
import ssl
import sys
import warnings
import zipfile

from pyeo import (
    classification, 
    filesystem_utilities,
    queries_and_downloads, 
    raster_manipulation,
    vectorisation
)
from pyeo.filesystem_utilities import config_to_log
from pyeo.acd_national import (
    acd_initialisation,
    acd_config_to_log,
    acd_roi_tile_intersection
)

gdal.UseExceptions()
warnings.simplefilter("ignore", category=UserWarning)

print("Libraries successfully imported")

/home/cmsstudent/miniconda3/envs/pyeo_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cmsstudent/miniconda3/envs/pyeo_env/lib/python3.10/abc.py:106: SHDeprecationWarning: AWS functionality will remain in the codebase for now, but won't be actively maintained.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


Libraries successfully imported


# make_composite.py

This App downloads and preprocesses Sentinel 2 images into a median image composite.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the above --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [4]:
%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-12-10 09:21:43,058: INFO: ---------------------------------------------------------------
2024-12-10 09:21:43,317: INFO: ---                 PROCESSING START                        ---
2024-12-10 09:21:43,318: INFO: ---------------------------------------------------------------
2024-12-10 09:21:43,319: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-12-10 09:21:43,319: INFO: True
2024-12-10 09:21:43,320: INFO: Config file that controls the processing run: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-12-10 09:21:43,321: INFO: ---------------------------------------------------------------
2024-12-10 09:21:43,323: INFO:   run_mode :  watch_period_seconds
2024-12-10 09:21:43,324: INFO:   forest_sentinel :  model
2024-12-10 09:21:43,325: INFO:   environment :  sen2cor_path
2024-12-10 09:21:43,326: INFO:   raster_processing_parameters :  change_to_classes
2024-12-10 09:21:43,326: INFO:   vector_processing_parameters :  minimum_area_to_report_m2
2

/home/cmsstudent/Desktop/pyeo_data/tilelist.csv already exists. Renaming the old file.
New file name: /home/cmsstudent/Desktop/pyeo_data/tilelist_5.csv


2024-12-10 09:21:46,867: INFO:     55 L1C products
2024-12-10 09:21:46,868: INFO:     42 L2A products
2024-12-10 09:21:46,869: INFO: --------------------------------------------------
2024-12-10 09:21:46,870: INFO: Filtering out L2A products that have the same 'start_date'  as another L2A product with a different processing baseline.  Dropping baseline N9999 where it is duplicating an existing baseline.
2024-12-10 09:21:46,944: INFO: --------------------------------------------------
2024-12-10 09:21:46,946: INFO: Filtering out L1C products that have the same 'beginposition' time stamp as an existing L2A product.
2024-12-10 09:21:46,947: INFO: Before filtering, 55 L1C and 42 L2A
2024-12-10 09:21:46,951: INFO: L1C : S2A_MSIL1C_20230101T075331_N0509_R135_T36MYE_20230101T093355.SAFE
2024-12-10 09:21:46,955: INFO: L1C : S2B_MSIL1C_20230106T075219_N0509_R135_T36MYE_20230106T093416.SAFE
2024-12-10 09:21:46,956: INFO: L1C : S2B_MSIL1C_20230116T075149_N0509_R135_T36MYE_20230116T093503.SAFE
202

runtime analysis saved to /home/cmsstudent/Desktop/pyeo_data/make_composite_2.prof
Run snakeviz over the profile file to interact with the profile information:
>snakeviz /home/cmsstudent/Desktop/pyeo_data/make_composite_2.prof


# detect_change.py
This App downloads and preprocesses Sentinel 2 images for change detection and classifies them using a machine learning model. It performs change detection against a baseline median image composite. Generates a report image file and optionally vectorises it if selected in the ini file.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [5]:
# navigate to the pyeo root directory
os.chdir(pyeo_dir)

In [6]:
%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-12-10 09:26:11,630: INFO: ---------------------------------------------------------------
2024-12-10 09:26:11,631: INFO: ---                 PROCESSING START                        ---
2024-12-10 09:26:11,632: INFO: ---------------------------------------------------------------
2024-12-10 09:26:11,633: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-12-10 09:26:11,634: INFO: ---------------------------------------------------------------
2024-12-10 09:26:11,636: INFO: Config file: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-12-10 09:26:11,636: INFO: ---------------------------------------------------------------
2024-12-10 09:26:11,637: INFO: ----------------------------
2024-12-10 09:26:11,638: INFO: Contents of the config file:
2024-12-10 09:26:11,639: INFO: ----------------------------
2024-12-10 09:26:11,640: INFO:   run_mode :  watch_period_seconds
2024-12-10 09:26:11,640: INFO:   forest_sentinel :  model
2024-12-10 09:26:11,641: INFO:

/home/cmsstudent/Desktop/pyeo_data/tilelist.csv already exists. Renaming the old file.
New file name: /home/cmsstudent/Desktop/pyeo_data/tilelist_6.csv


2024-12-10 09:26:12,601: INFO: Removed 0 faulty scenes <0MB in size from the list
2024-12-10 09:26:12,612: INFO:     15 L1C products
2024-12-10 09:26:12,613: INFO:     10 L2A products
2024-12-10 09:26:12,615: INFO: Capping the number of L1C products to 3
2024-12-10 09:26:12,617: INFO: Relative orbits found covering tile: [135]
2024-12-10 09:26:12,622: INFO:     3 L1C products remain:
2024-12-10 09:26:12,624: INFO:        S2A_MSIL1C_20240912T080601_N0511_R135_T36MYE_20240912T100522.SAFE
2024-12-10 09:26:12,625: INFO:        S2A_MSIL1C_20240922T074631_N0511_R135_T36MYE_20240922T094322.SAFE
2024-12-10 09:26:12,626: INFO:        S2A_MSIL1C_20241012T074841_N0511_R135_T36MYE_20241012T093627.SAFE
2024-12-10 09:26:12,627: INFO: Number of L1C products for dataspace is 3
2024-12-10 09:26:12,629: INFO: Capping the number of L2A products to 3
2024-12-10 09:26:12,630: INFO: Relative orbits found covering tile: [135]
2024-12-10 09:26:12,635: INFO:     3 L2A products remain:
2024-12-10 09:26:12,637: 

runtime analysis saved to /home/cmsstudent/Desktop/pyeo_data/detect_change_1.prof
Run snakeviz over the profile file to interact with the profile information:
>snakeviz /home/cmsstudent/Desktop/pyeo_data/detect_change_1.prof


# send_report.py
This App sends out any new vectorised report files (.shp format) to a list of recipients specified in an email list file. Currently only Gmail is implemented. WhatsApp will be added in the future. Options are set in the .ini file and login details in the credentials file. Any newly created shapefiles found will be zipped and emailed out as an attachment. The zipping process ensures that they are not sent out twice.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [7]:
# navigate to the pyeo root directory
os.chdir(pyeo_dir)

In [8]:
%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-12-10 10:53:27,297: INFO: ---------------------------------------------------------------
2024-12-10 10:53:27,300: INFO: ---                 PROCESSING START                        ---
2024-12-10 10:53:27,303: INFO: ---------------------------------------------------------------
2024-12-10 10:53:27,305: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-12-10 10:53:27,306: INFO: True
2024-12-10 10:53:27,307: INFO: Config file that controls the processing run: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-12-10 10:53:27,308: INFO: ---------------------------------------------------------------
2024-12-10 10:53:27,309: INFO: ----------------------------
2024-12-10 10:53:27,310: INFO: Contents of the config file:
2024-12-10 10:53:27,311: INFO: ----------------------------
2024-12-10 10:53:27,312: INFO:   run_mode :  watch_period_seconds
2024-12-10 10:53:27,313: INFO:   forest_sentinel :  model
2024-12-10 10:53:27,315: INFO:   environment :  sen2cor_

/home/cmsstudent/Desktop/pyeo_data/tilelist.csv already exists. Renaming the old file.
New file name: /home/cmsstudent/Desktop/pyeo_data/tilelist_7.csv


2024-12-10 10:53:31,273: INFO:  
2024-12-10 10:53:31,275: INFO: ---------------------------------------------------------------
2024-12-10 10:53:31,277: INFO: ---             TILE PROCESSING END                           ---
2024-12-10 10:53:31,278: INFO: ---------------------------------------------------------------
2024-12-10 10:53:31,279: INFO:  
2024-12-10 10:53:31,284: INFO: ---------------------------------------------------------------
2024-12-10 10:53:31,285: INFO: ---                  ALL PROCESSING END                      ---
2024-12-10 10:53:31,286: INFO: ---------------------------------------------------------------


runtime analysis saved to /home/cmsstudent/Desktop/pyeo_data/send_report_1.prof
Run snakeviz over the profile file to interact with the profile information:
>snakeviz /home/cmsstudent/Desktop/pyeo_data/send_report_1.prof
